In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

# Load historical data
df = pd.read_csv('../Data/cleaned_data.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Define the backtesting period t
start_date = '2024-08-01'
end_date = '2025-07-31'

# Filter data for the backtesting period
df_backtest = df[(df.index >= start_date) & (df.index <= end_date)]

# Check the columns in df_backtest
print("Columns in df_backtest:", df_backtest.columns)

# Define optimal weights from Task 4 (example values)
optimal_weights = {'TSLA': 0.5, 'BND': 0.2, 'SPY': 0.3}  # Replace with your actual weights

# Fetch historical data for BND and SPY for the backtesting period
assets = ['BND', 'SPY']
benchmark_data = yf.download(assets, start=start_date, end=end_date)['Adj Close']

# Calculate daily returns for the benchmark
benchmark_returns = benchmark_data.pct_change().dropna()

# Verify that the necessary columns exist in df_backtest
if 'BND' in df_backtest.columns and 'SPY' in df_backtest.columns and 'TSLA' in df_backtest.columns:
    # Calculate strategy returns
    strategy_returns = (df_backtest['BND'].pct_change() * optimal_weights['BND'] +
                        df_backtest['SPY'].pct_change() * optimal_weights['SPY'] +
                        df_backtest['TSLA'].pct_change() * optimal_weights['TSLA'])
else:
    print("Ensure df_backtest contains the columns: 'BND', 'SPY', 'TSLA'")

# Calculate cumulative returns
cumulative_strategy = (1 + strategy_returns).cumprod() - 1
cumulative_benchmark = (1 + (benchmark_returns['BND'] * 0.4 + benchmark_returns['SPY'] * 0.6)).cumprod() - 1

# Plotting the cumulative returns
plt.figure(figsize=(14, 7))
plt.plot(cumulative_strategy, label='Strategy Portfolio', color='orange')
plt.plot(cumulative_benchmark, label='Benchmark Portfolio', color='blue')
plt.title('Cumulative Returns: Strategy vs Benchmark')
plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
plt.legend()
plt.grid()
plt.show()

# Performance Metrics: Total Return and Sharpe Ratio
final_strategy_return = cumulative_strategy.iloc[-1]
final_benchmark_return = cumulative_benchmark.iloc[-1]

# Annualized Sharpe Ratio calculation
risk_free_rate = 0.0175  # Example: 1.75% risk-free rate
strategy_sharpe = (strategy_returns.mean() * 252 - risk_free_rate) / (strategy_returns.std() * np.sqrt(252))
benchmark_sharpe = ((benchmark_returns['BND'].mean() * 0.4 + benchmark_returns['SPY'].mean() * 0.6) * 252 - risk_free_rate) / \
                    np.sqrt(((benchmark_returns['BND'].std() * 0.4)**2 + (benchmark_returns['SPY'].std() * 0.6)**2) / 2) * 252)

# Summary of results
print(f"Final Strategy Total Return: {final_strategy_return:.2%}")
print(f"Final Benchmark Total Return: {final_benchmark_return:.2%}")
print(f"Strategy Sharpe Ratio: {strategy_sharpe:.2f}")
print(f"Benchmark Sharpe Ratio: {benchmark_sharpe:.2f}")

# Conclusion
if final_strategy_return > final_benchmark_return:
    conclusion = "Your strategy outperformed the benchmark."
else:
    conclusion = "Your strategy underperformed the benchmark."

print(conclusion)